# Automated Machine Learning on Azure
In this file, I train multiple machine learning models, automated with Azure.

In [ ]:
pip show azure-ai-ml

In [1]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

My credentials, subscription id, resource group and workspace info, are all stored in a file called config.py. You have to make sure that you have yours in that file.

In [2]:
import config

In [3]:
ml_client = MLClient(
    DefaultAzureCredential(), config.subscription_id, config.resource_group, config.workspace
)

In [4]:
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes, InputOutputModes

data_asset = ml_client.data.get("titanic_ds5", version="1")

# creates a dataset 
my_training_data_input = Input(path=data_asset.id,
                type=AssetTypes.MLTABLE,
                mode=InputOutputModes.RO_MOUNT
            )

In [5]:
from azure.ai.ml import automl

# configure the classification job
classification_job = automl.classification(
    compute="sckaraman1",
    experiment_name="titanic_autoML_classification",
    training_data=my_training_data_input,
    target_column_name="Survived",
    primary_metric="accuracy",
    n_cross_validations=5,
    enable_model_explainability=True
)

# set the limits
classification_job.set_limits(
    timeout_minutes=60, 
    trial_timeout_minutes=20, 
    max_trials=5,
    enable_early_termination=True,
)

In [6]:
# Submit the AutoML job
returned_job = ml_client.jobs.create_or_update(
    classification_job
)  

# submit the job to the backend
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

Monitor your job at https://ml.azure.com/runs/placid_lizard_m2l7r7vpx7?wsid=/subscriptions/a54b1e51-86a2-4073-b2a5-1a79c43cf955/resourcegroups/model_dep/workspaces/ml-workspace&tid=2e5df792-f2e7-43fc-954b-c3dd8d56a02d
